# 📊 Item (d): Determinação de Kp para Overshoot de 15%

## 🚢 Veículo Submersível Não-Tripulado

### 📋 Descrição do Problema

Considere um veículo submersível não-tripulado cuja planta que relaciona o ângulo do leme de profundidade, ψ, e o ângulo de arfagem, θ, tenha Função Transferência (FT) da forma:

$$\hat{\theta}(s) / \hat{\psi}(s) = \hat{g}(s) = -\frac{0,25s + 0,10875}{s^4 + 3,456s^3 + 3,45688s^2 + 0,719297s + 0,041574}$$

### 🎯 Objetivo do Exercício (d)

**Na situação anterior (item c), determine Kp que faz o overshoot ser igual a 15%. Qual o tempo de acomodação esperado?**

### 📝 Metodologia

1. **Relação Overshoot-Amortecimento**: Usar fórmula $M_p = e^{\left( \frac{-\zeta\pi}{\sqrt{1-\zeta^2}} \right)}$
2. **Calcular ζ**: Resolver equação para Mp = 15% = 0.15
3. **Determinar ωn**: Usar polinômio característico de malha fechada
4. **Encontrar Kp**: Comparar com forma padrão $s^2 + 2\zeta\omega_n s + \omega_n^2$
5. **Calcular ts**: Usar critérios de 2% e 5% para tempo de acomodação
6. **Validação**: Verificar se Kp está na faixa válida do item (c)

### 🔍 Conceitos Teóricos

**Overshoot (Mp)**: Máximo valor que a resposta ultrapassa o valor final  
**Coeficiente de Amortecimento (ζ)**: Parâmetro que caracteriza o tipo de resposta  
**Frequência Natural (ωn)**: Frequência de oscilação natural do sistema  
**Tempo de Acomodação (ts)**: Tempo para a resposta se manter dentro de uma faixa em torno do valor final

In [12]:
# 🔧 1. Configuração e Importações
# Importação das bibliotecas necessárias para análise de resposta transitória e cálculo de overshoot.

In [13]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import lti, step
from scipy.optimize import fsolve
import warnings
warnings.filterwarnings('ignore')

# Verificar se sympy está disponível para LaTeX
try:
    import sympy as sp
    from sympy import symbols, latex, simplify, solve, log, exp, pi, sqrt
    from IPython.display import display, Math
    sympy_available = True
    print("✅ Biblioteca 'sympy' disponível para renderização LaTeX")
except ImportError:
    sympy_available = False
    print("❌ Biblioteca 'sympy' não disponível")

# Configuração dos gráficos
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11
plt.rcParams['grid.alpha'] = 0.3

print("📦 Bibliotecas importadas com sucesso!")

✅ Biblioteca 'sympy' disponível para renderização LaTeX
📦 Bibliotecas importadas com sucesso!


In [14]:
# 📐 2. Definição de Funções Auxiliares
# Funções para análise de overshoot e características de resposta transitória.

In [15]:
def definir_sistema_reduzido():
    """
    Define o sistema reduzido de 2ª ordem obtido no item (b).
    
    G_red(s) = -0.2533 / (s² + 0.216s + 0.0969)
    
    Returns:
        scipy.signal.lti: Sistema reduzido de 2ª ordem
    """
    numerador = [-0.2533]
    denominador = [1, 0.216, 0.0969]
    sistema_reduzido = lti(numerador, denominador)
    return sistema_reduzido

def calcular_zeta_de_overshoot(Mp):
    """
    Calcula o coeficiente de amortecimento ζ a partir do overshoot.
    
    Fórmula: Mp = exp(-ζπ/√(1-ζ²))
    Resolvendo para ζ: ζ = √[ln²(Mp)/(π² + ln²(Mp))]
    
    Args:
        Mp (float): Overshoot em formato decimal (0.15 para 15%)
    
    Returns:
        float: Coeficiente de amortecimento ζ
    """
    if Mp <= 0 or Mp >= 1:
        raise ValueError("Overshoot deve estar entre 0 e 1 (0% e 100%)")
    
    ln_Mp = np.log(Mp)
    zeta = np.sqrt((ln_Mp**2) / (np.pi**2 + ln_Mp**2))
    
    return zeta

def calcular_parametros_segunda_ordem(zeta, coef_2zeta_wn):
    """
    Calcula ωn e outros parâmetros do sistema de 2ª ordem.
    
    Args:
        zeta (float): Coeficiente de amortecimento
        coef_2zeta_wn (float): Coeficiente 2ζωn do polinômio característico
    
    Returns:
        tuple: (omega_n, omega_d, sigma, Tp, Ts_2pct, Ts_5pct)
    """
    # Frequência natural
    omega_n = coef_2zeta_wn / (2 * zeta)
    
    # Frequência natural amortecida
    omega_d = omega_n * np.sqrt(1 - zeta**2)
    
    # Constante de tempo (parte real dos polos)
    sigma = zeta * omega_n
    
    # Tempo de pico
    Tp = np.pi / omega_d
    
    # Tempo de acomodação (critério 2%)
    Ts_2pct = 4 / sigma
    
    # Tempo de acomodação (critério 5%)
    Ts_5pct = 3 / sigma
    
    return omega_n, omega_d, sigma, Tp, Ts_2pct, Ts_5pct

def calcular_kp_para_overshoot(Mp_desejado):
    """
    Calcula o ganho Kp necessário para obter um overshoot específico.
    
    Args:
        Mp_desejado (float): Overshoot desejado em formato decimal
    
    Returns:
        tuple: (kp, zeta, omega_n, parametros_completos)
    """
    # Passo 1: Calcular ζ a partir do overshoot
    zeta = calcular_zeta_de_overshoot(Mp_desejado)
    
    # Coeficiente 2ζωn do sistema (do item c: 0.216)
    coef_2zeta_wn = 0.216
    
    # Passo 2: Calcular parâmetros do sistema
    omega_n, omega_d, sigma, Tp, Ts_2pct, Ts_5pct = calcular_parametros_segunda_ordem(zeta, coef_2zeta_wn)
    
    # Passo 3: Calcular Kp
    # Do polinômio característico: s² + 0.216s + (0.0969 - 0.2533*Kp)
    # Forma padrão: s² + 2ζωns + ωn²
    # Portanto: ωn² = 0.0969 - 0.2533*Kp
    # Kp = (0.0969 - ωn²) / 0.2533
    
    omega_n_squared = omega_n**2
    kp = (0.0969 - omega_n_squared) / 0.2533
    
    parametros = {
        'zeta': zeta,
        'omega_n': omega_n,
        'omega_d': omega_d,
        'sigma': sigma,
        'Tp': Tp,
        'Ts_2pct': Ts_2pct,
        'Ts_5pct': Ts_5pct,
        'Mp': Mp_desejado,
        'kp': kp
    }
    
    return kp, zeta, omega_n, parametros

def verificar_overshoot_simulacao(kp):
    """
    Verifica o overshoot através de simulação da resposta ao degrau.
    
    Args:
        kp (float): Ganho proporcional
    
    Returns:
        tuple: (overshoot_percentual, tempo_pico, valor_pico, valor_final)
    """
    # Sistema reduzido
    sistema_reduzido = definir_sistema_reduzido()
    
    # Malha fechada
    num_mf = np.array(sistema_reduzido.num) * kp
    den_mf = np.array(sistema_reduzido.den) + np.pad(num_mf, (len(sistema_reduzido.den) - len(num_mf), 0), 'constant')
    
    sistema_mf = lti(num_mf, den_mf)
    
    # Simular resposta ao degrau
    t, y = step(sistema_mf)
    
    # Encontrar valor final
    valor_final = y[-1]
    
    # Encontrar pico
    indice_pico = np.argmax(np.abs(y))
    valor_pico = y[indice_pico]
    tempo_pico = t[indice_pico]
    
    # Calcular overshoot
    if valor_final != 0:
        overshoot = abs((valor_pico - valor_final) / valor_final)
    else:
        overshoot = 0
    
    return overshoot, tempo_pico, valor_pico, valor_final, t, y

In [16]:
# 🧮 3. Análise Matemática para Overshoot de 15%
# Cálculo rigoroso do ganho Kp para overshoot de 15% e determinação do tempo de acomodação.

In [17]:
def analise_matematica_completa():
    """
    Realiza a análise matemática completa para determinar Kp
    que resulta em overshoot de 15%.
    """
    print("🔍 ANÁLISE MATEMÁTICA DETALHADA")
    print("=" * 80)
    
    # Especificações do problema
    Mp_desejado = 0.15  # 15% overshoot
    
    print("1. ESPECIFICAÇÕES DO PROBLEMA:")
    print(f"   Overshoot desejado: {Mp_desejado*100:.1f}%")
    print("   Sistema: Malha fechada com controlador proporcional Kp")
    print("   Base: Sistema reduzido de 2ª ordem (item b)")
    
    print("\n2. RELAÇÃO OVERSHOOT-AMORTECIMENTO:")
    print("   Para sistema de 2ª ordem subamortecido:")
    print("   Mp = exp(-ζπ/√(1-ζ²))")
    print("   Resolvendo para ζ:")
    print("   ζ = √[ln²(Mp)/(π² + ln²(Mp))]")
    
    # Calcular ζ
    zeta = calcular_zeta_de_overshoot(Mp_desejado)
    ln_Mp = np.log(Mp_desejado)
    
    print(f"\n   Substituindo Mp = {Mp_desejado}:")
    print(f"   ln(Mp) = ln({Mp_desejado}) = {ln_Mp:.4f}")
    print(f"   ζ = √[({ln_Mp:.4f})²/(π² + ({ln_Mp:.4f})²)]")
    print(f"   ζ = √[{ln_Mp**2:.4f}/({np.pi**2:.4f} + {ln_Mp**2:.4f})]")
    print(f"   ζ = √[{ln_Mp**2:.4f}/{np.pi**2 + ln_Mp**2:.4f}]")
    print(f"   ζ = {zeta:.4f}")
    
    print("\n3. POLINÔMIO CARACTERÍSTICO:")
    print("   Do item (c): P(s) = s² + 0.216s + (0.0969 - 0.2533*Kp)")
    print("   Forma padrão: s² + 2ζωₙs + ωₙ²")
    print("   Comparando coeficientes:")
    print("   • 2ζωₙ = 0.216")
    print("   • ωₙ² = 0.0969 - 0.2533*Kp")
    
    # Calcular ωₙ
    omega_n = 0.216 / (2 * zeta)
    omega_n_squared = omega_n**2
    
    print(f"\n4. CÁLCULO DA FREQUÊNCIA NATURAL:")
    print(f"   ωₙ = 0.216/(2ζ) = 0.216/(2×{zeta:.4f})")
    print(f"   ωₙ = 0.216/{2*zeta:.4f} = {omega_n:.4f} rad/s")
    print(f"   ωₙ² = ({omega_n:.4f})² = {omega_n_squared:.4f}")
    
    # Calcular Kp
    kp = (0.0969 - omega_n_squared) / 0.2533
    
    print(f"\n5. CÁLCULO DO GANHO PROPORCIONAL:")
    print(f"   Da segunda relação: ωₙ² = 0.0969 - 0.2533*Kp")
    print(f"   {omega_n_squared:.4f} = 0.0969 - 0.2533*Kp")
    print(f"   0.2533*Kp = 0.0969 - {omega_n_squared:.4f}")
    print(f"   0.2533*Kp = {0.0969 - omega_n_squared:.4f}")
    print(f"   Kp = {0.0969 - omega_n_squared:.4f}/0.2533")
    print(f"   Kp = {kp:.4f}")
    
    # Calcular parâmetros de resposta
    omega_n, omega_d, sigma, Tp, Ts_2pct, Ts_5pct = calcular_parametros_segunda_ordem(zeta, 0.216)
    
    print(f"\n6. PARÂMETROS DE RESPOSTA TRANSITÓRIA:")
    print(f"   • Frequência natural: ωₙ = {omega_n:.4f} rad/s")
    print(f"   • Frequência amortecida: ωₐ = {omega_d:.4f} rad/s")
    print(f"   • Constante de tempo: σ = ζωₙ = {sigma:.4f}")
    print(f"   • Tempo de pico: Tp = π/ωₐ = {Tp:.2f} s")
    
    print(f"\n7. TEMPO DE ACOMODAÇÃO:")
    print(f"   • Critério 2%: ts = 4/σ = 4/{sigma:.4f} = {Ts_2pct:.2f} s")
    print(f"   • Critério 5%: ts = 3/σ = 3/{sigma:.4f} = {Ts_5pct:.2f} s")
    
    # Verificar se está na faixa válida do item (c)
    faixa_valida = 0 < kp < 0.3365
    print(f"\n8. VALIDAÇÃO:")
    print(f"   Faixa válida do item (c): 0 < Kp < 0.3365")
    print(f"   Kp calculado = {kp:.4f}")
    print(f"   Status: {'✓ VÁLIDO' if faixa_valida else '✗ INVÁLIDO'}")
    
    return kp, zeta, omega_n, {
        'zeta': zeta,
        'omega_n': omega_n,
        'omega_d': omega_d,
        'sigma': sigma,
        'Tp': Tp,
        'Ts_2pct': Ts_2pct,
        'Ts_5pct': Ts_5pct,
        'Mp': Mp_desejado,
        'kp': kp
    }

# Executar análise
kp_calculado, zeta_calc, omega_n_calc, parametros = analise_matematica_completa()

🔍 ANÁLISE MATEMÁTICA DETALHADA
1. ESPECIFICAÇÕES DO PROBLEMA:
   Overshoot desejado: 15.0%
   Sistema: Malha fechada com controlador proporcional Kp
   Base: Sistema reduzido de 2ª ordem (item b)

2. RELAÇÃO OVERSHOOT-AMORTECIMENTO:
   Para sistema de 2ª ordem subamortecido:
   Mp = exp(-ζπ/√(1-ζ²))
   Resolvendo para ζ:
   ζ = √[ln²(Mp)/(π² + ln²(Mp))]

   Substituindo Mp = 0.15:
   ln(Mp) = ln(0.15) = -1.8971
   ζ = √[(-1.8971)²/(π² + (-1.8971)²)]
   ζ = √[3.5991/(9.8696 + 3.5991)]
   ζ = √[3.5991/13.4687]
   ζ = 0.5169

3. POLINÔMIO CARACTERÍSTICO:
   Do item (c): P(s) = s² + 0.216s + (0.0969 - 0.2533*Kp)
   Forma padrão: s² + 2ζωₙs + ωₙ²
   Comparando coeficientes:
   • 2ζωₙ = 0.216
   • ωₙ² = 0.0969 - 0.2533*Kp

4. CÁLCULO DA FREQUÊNCIA NATURAL:
   ωₙ = 0.216/(2ζ) = 0.216/(2×0.5169)
   ωₙ = 0.216/1.0339 = 0.2089 rad/s
   ωₙ² = (0.2089)² = 0.0436

5. CÁLCULO DO GANHO PROPORCIONAL:
   Da segunda relação: ωₙ² = 0.0969 - 0.2533*Kp
   0.0436 = 0.0969 - 0.2533*Kp
   0.2533*Kp = 0.0969 -

In [18]:
## 📝 4. Apresentação Matemática em LaTeX

In [23]:
def apresentacao_latex_completa():
    """
    Apresenta toda a análise matemática usando LaTeX formatado.
    """
    print("📝 APRESENTAÇÃO MATEMÁTICA EM LaTeX")
    print("=" * 80)
    
    if not sympy_available:
        print("⚠️ Sympy não disponível - usando apresentação em texto")
        return
    
    # Especificação do problema
    display(Math(r'\text{Especificação: Overshoot } M_p = 15\% = 0.15'))
    
    # Relação overshoot-amortecimento
    display(Math(r'\text{Relação Overshoot-Amortecimento:}'))
    display(Math(r'M_p = e^{\left( \frac{-\zeta\pi}{\sqrt{1-\zeta^2}} \right)}'))
    
    display(Math(r'\text{Resolvendo para } \zeta \text{:}'))
    display(Math(r'\ln(M_p) = -\frac{\zeta\pi}{\sqrt{1-\zeta^2}}'))
    
    display(Math(r'\zeta = \sqrt{\frac{\ln^2(M_p)}{\pi^2 + \ln^2(M_p)}}'))
    
    # Substituição numérica
    display(Math(r'\text{Substituindo } M_p = 0.15\text{:}'))
    display(Math(r'\ln(0.15) = -1.897'))
    display(Math(r'\zeta = \sqrt{\frac{(-1.897)^2}{\pi^2 + (-1.897)^2}} = \sqrt{\frac{3.598}{9.870 + 3.598}} = 0.517'))
    
    # Polinômio característico
    display(Math(r'\text{Polinômio Característico (item c):}'))
    display(Math(r'P(s) = s^2 + 0.216s + (0.0969 - 0.2533 K_p)'))
    
    display(Math(r'\text{Forma Padrão:}'))
    display(Math(r's^2 + 2\zeta\omega_n s + \omega_n^2'))
    
    # Comparação de coeficientes
    display(Math(r'\text{Comparando coeficientes:}'))
    display(Math(r'2\zeta\omega_n = 0.216'))
    display(Math(r'\omega_n^2 = 0.0969 - 0.2533 K_p'))
    
    # Cálculo de ωn
    display(Math(r'\text{Da primeira relação:}'))
    display(Math(r'\omega_n = \frac{0.216}{2\zeta} = \frac{0.216}{2 \times 0.517} = 0.2089 \text{ rad/s}'))
    
    # Cálculo de Kp
    display(Math(r'\text{Da segunda relação:}'))
    display(Math(r'\omega_n^2 = (0.2089)^2 = 0.0436'))
    display(Math(r'0.0436 = 0.0969 - 0.2533 K_p'))
    display(Math(r'0.2533 K_p = 0.0969 - 0.0436 = 0.0533'))
    display(Math(r'K_p = \frac{0.0533}{0.2533} = 0.2104'))
    
    # Tempo de acomodação
    display(Math(r'\text{Tempo de Acomodação:}'))
    display(Math(r'\sigma = \zeta\omega_n = 0.517 \times 0.2089 = 0.108'))
    
    display(Math(r'\text{Critério 5\%: } t_s = \frac{3}{\sigma} = \frac{3}{0.108} = 27.8 \text{ s}'))
    display(Math(r'\text{Critério 2\%: } t_s = \frac{4}{\sigma} = \frac{4}{0.108} = 37.0 \text{ s}'))
    
    # Resultado final
    display(Math(r'\text{Resultados Finais:}'))
    display(Math(r'\boxed{K_p = 0.2104}'))
    display(Math(r'\boxed{t_s = 27.8 \text{ s (5\%)} \text{ ou } 37.0 \text{ s (2\%)}}'))

# Apresentar em LaTeX
apresentacao_latex_completa()

📝 APRESENTAÇÃO MATEMÁTICA EM LaTeX


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## 📊 5. Verificação e Simulação Numérica

In [20]:
# 📊 5. Verificação Numérica e Validação

print("🔍 VERIFICAÇÃO NUMÉRICA COMPLETA")
print("=" * 80)

# Executar simulação com o Kp calculado
overshoot_sim, tempo_pico_sim, valor_pico, valor_final, t, y = verificar_overshoot_simulacao(parametros['kp'])

print(f"📈 RESULTADOS DA SIMULAÇÃO:")
print(f"   • Overshoot simulado: {overshoot_sim*100:.2f}% (objetivo: 15.0%)")
print(f"   • Tempo de pico: {tempo_pico_sim:.2f} s (teórico: {parametros['Tp']:.2f} s)")
print(f"   • Valor de pico: {valor_pico:.4f}")
print(f"   • Valor final: {valor_final:.4f}")
print(f"   • Erro no overshoot: {abs(0.15 - overshoot_sim)*100:.3f}%")

# Validação matemática
print(f"\n✅ VALIDAÇÃO:")
print(f"   • Overshoot dentro da tolerância: {'✓' if abs(0.15 - overshoot_sim) < 0.01 else '✗'}")
print(f"   • Kp na faixa válida (0 < Kp < 0.3365): {'✓' if 0 < parametros['kp'] < 0.3365 else '✗'}")
print(f"   • Sistema subamortecido: {'✓' if parametros['zeta'] < 1 else '✗'}")

# Salvar valores para uso posterior
overshoot_verificado = overshoot_sim
tempo_pico_verificado = tempo_pico_sim

print(f"\n🎯 RESULTADO FINAL:")
print(f"   Kp para overshoot de 15%: {parametros['kp']:.4f}")
print(f"   Overshoot obtido: {overshoot_verificado*100:.2f}%")

🔍 VERIFICAÇÃO NUMÉRICA COMPLETA
📈 RESULTADOS DA SIMULAÇÃO:
   • Overshoot simulado: 15.00% (objetivo: 15.0%)
   • Tempo de pico: 17.68 s (teórico: 17.57 s)
   • Valor de pico: -1.4029
   • Valor final: -1.2199
   • Erro no overshoot: 0.003%

✅ VALIDAÇÃO:
   • Overshoot dentro da tolerância: ✓
   • Kp na faixa válida (0 < Kp < 0.3365): ✓
   • Sistema subamortecido: ✓

🎯 RESULTADO FINAL:
   Kp para overshoot de 15%: 0.2102
   Overshoot obtido: 15.00%


## 🎯 6. Conclusões e Resultados Finais

In [21]:
print("🎯 CONCLUSÕES FINAIS - ITEM (d)")
print("=" * 80)

print("\n📋 RESUMO DA ANÁLISE:")
print("-" * 50)
print("✓ Objetivo: Determinar Kp para overshoot de 15%")
print("✓ Sistema: Malha fechada com sistema reduzido de 2ª ordem")
print("✓ Base teórica: Relação Mp = exp(-ζπ/√(1-ζ²))")
print("✓ Método: Comparação com forma padrão de 2ª ordem")

print("\n🧮 DESENVOLVIMENTO MATEMÁTICO:")
print("-" * 50)
print(f"• Overshoot especificado: 15%")
print(f"• Coeficiente de amortecimento: ζ = {parametros['zeta']:.4f}")
print(f"• Frequência natural: ωₙ = {parametros['omega_n']:.4f} rad/s")
print(f"• Polinômio característico: s² + 0.216s + (0.0969 - 0.2533*Kp)")
print(f"• Comparação: 2ζωₙ = 0.216, ωₙ² = 0.0969 - 0.2533*Kp")

print("\n📊 RESULTADOS PRINCIPAIS:")
print("-" * 50)
print("1. GANHO PROPORCIONAL:")
print(f"   📌 Kp = {parametros['kp']:.4f}")

print("\n2. TEMPO DE ACOMODAÇÃO:")
print(f"   📌 Critério 5%: ts = {parametros['Ts_5pct']:.1f} segundos")
print(f"   📌 Critério 2%: ts = {parametros['Ts_2pct']:.1f} segundos")

print("\n3. OUTROS PARÂMETROS:")
print(f"   • Tempo de pico: {parametros['Tp']:.1f} s")
print(f"   • Frequência amortecida: {parametros['omega_d']:.4f} rad/s")
print(f"   • Constante de tempo: σ = {parametros['sigma']:.4f}")

print("\n✅ VERIFICAÇÃO NUMÉRICA:")
print("-" * 50)
print(f"• Overshoot teórico: 15.0%")
print(f"• Overshoot simulado: {overshoot_verificado*100:.1f}%")
print(f"• Erro absoluto: {abs(0.15 - overshoot_verificado)*100:.2f}%")
print(f"• Tempo de pico teórico: {parametros['Tp']:.1f} s")
print(f"• Tempo de pico simulado: {tempo_pico_verificado:.1f} s")

print("\n🔍 VALIDAÇÕES:")
print("-" * 50)
faixa_valida = 0 < parametros['kp'] < 0.3365
print(f"• Faixa válida do item (c): 0 < Kp < 0.3365")
print(f"• Kp calculado: {parametros['kp']:.4f}")
print(f"• Status: {'✓ DENTRO DA FAIXA VÁLIDA' if faixa_valida else '✗ FORA DA FAIXA VÁLIDA'}")
print(f"• Comportamento: {'✓ SUBAMORTECIDO' if faixa_valida else '✗ NÃO SUBAMORTECIDO'}")
print(f"• Sistema: {'✓ ESTÁVEL' if faixa_valida else '✗ INSTÁVEL'}")

print("\n📈 CARACTERÍSTICAS DA RESPOSTA:")
print("-" * 50)
print("• Resposta oscilatória com envelope decrescente")
print(f"• Overshoot de exatamente 15%")
print(f"• {parametros['Tp']:.1f} oscilações por minuto aproximadamente")
print(f"• Acomodação em {parametros['Ts_5pct']:.0f}-{parametros['Ts_2pct']:.0f} segundos")

print("\n⚠️ OBSERVAÇÕES IMPORTANTES:")
print("-" * 50)
print("• Análise baseada no sistema reduzido de 2ª ordem")
print("• Configuração de malha fechada com realimentação unitária")
print("• Controlador puramente proporcional (sem integral ou derivativo)")
print("• Resultados válidos para pequenas perturbações em torno do ponto de operação")

print("\n" + "=" * 80)
print("📝 RESPOSTA AO ITEM (d):")
print("")
print("🎯 O ganho proporcional que resulta em overshoot de 15% é:")
print(f"                    Kp = {parametros['kp']:.4f}")
print("")
print("⏱️ O tempo de acomodação esperado é:")
print(f"    • Critério 5%: {parametros['Ts_5pct']:.1f} segundos")
print(f"    • Critério 2%: {parametros['Ts_2pct']:.1f} segundos")
print("")
print("=" * 80)

🎯 CONCLUSÕES FINAIS - ITEM (d)

📋 RESUMO DA ANÁLISE:
--------------------------------------------------
✓ Objetivo: Determinar Kp para overshoot de 15%
✓ Sistema: Malha fechada com sistema reduzido de 2ª ordem
✓ Base teórica: Relação Mp = exp(-ζπ/√(1-ζ²))
✓ Método: Comparação com forma padrão de 2ª ordem

🧮 DESENVOLVIMENTO MATEMÁTICO:
--------------------------------------------------
• Overshoot especificado: 15%
• Coeficiente de amortecimento: ζ = 0.5169
• Frequência natural: ωₙ = 0.2089 rad/s
• Polinômio característico: s² + 0.216s + (0.0969 - 0.2533*Kp)
• Comparação: 2ζωₙ = 0.216, ωₙ² = 0.0969 - 0.2533*Kp

📊 RESULTADOS PRINCIPAIS:
--------------------------------------------------
1. GANHO PROPORCIONAL:
   📌 Kp = 0.2102

2. TEMPO DE ACOMODAÇÃO:
   📌 Critério 5%: ts = 27.8 segundos
   📌 Critério 2%: ts = 37.0 segundos

3. OUTROS PARÂMETROS:
   • Tempo de pico: 17.6 s
   • Frequência amortecida: 0.1788 rad/s
   • Constante de tempo: σ = 0.1080

✅ VERIFICAÇÃO NUMÉRICA:
---------------

In [22]:
# Teste simples para verificar o resultado
print("🎯 RESUMO DO ITEM (d)")
print("=" * 50)
print(f"Kp calculado: {parametros['kp']:.4f}")
print(f"Overshoot esperado: 15.0%")
print(f"Tempo de acomodação (5%): {parametros['Ts_5pct']:.1f} s")
print(f"Tempo de acomodação (2%): {parametros['Ts_2pct']:.1f} s")

🎯 RESUMO DO ITEM (d)
Kp calculado: 0.2102
Overshoot esperado: 15.0%
Tempo de acomodação (5%): 27.8 s
Tempo de acomodação (2%): 37.0 s
